## Initialize Maxim SDK

In [1]:
from typing import Dict, Optional
from dotenv import dotenv_values
from maxim import Config, Maxim
from maxim.evaluators import BaseEvaluator
from maxim.models import (
    LocalEvaluatorResultParameter,
    LocalEvaluatorReturn,
    ManualData,
    PassFailCriteria,
)
from maxim.models.evaluator import (
    PassFailCriteriaForTestrunOverall,
    PassFailCriteriaOnEachEntry,
)

config = dotenv_values()

API_KEY: str = config.get("MAXIM_API_KEY") or ""
WORKSPACE_ID: str = config.get("MAXIM_WORKSPACE_ID") or ""
WORKFLOW_ID: str = config.get("MAXIM_WORKFLOW_ID") or ""
DATASET_ID: str = config.get("MAXIM_DATASET_ID") or ""
PROMPT_VERSION_ID: str = config.get("MAXIM_PROMPT_VERSION_ID") or ""
MAXIM_UNKNOWN_WORKFLOW_ID: str = config.get("MAXIM_UNKNOWN_WORKFLOW_ID") or ""
MAXIM_INVALID_WORKFLOW_ID: str = config.get("MAXIM_INVALID_WORKFLOW_ID") or ""

maxim = Maxim(
    config=Config(
        api_key=API_KEY,
    )
)

## Define custom evaluator

In [2]:

class MyCustomEvaluator(BaseEvaluator):
    """
    Custom evaluator class that extends BaseEvaluator to perform custom evaluations.
    """

    def evaluate(
        self, result: LocalEvaluatorResultParameter, data: ManualData
    ) -> Dict[str, LocalEvaluatorReturn]:
        """
        You are supposed to override this function and run evaluations
        Args:
            result (LocalEvaluatorResultParameter): The result parameter containing evaluation data
            data (ManualData): The manual data to evaluate against
        Returns:
            Dict[str, LocalEvaluatorReturn]: Dictionary mapping evaluation names to score results.
        """
        return {
            "Evaluation 1": LocalEvaluatorReturn(score=1),
            "Evaluation 2": LocalEvaluatorReturn(score=False, reasoning="Just chillll"),
        }

## Create and trigger test run

In [3]:
maxim.create_test_run(
    name="Local evaluators from SDK", in_workspace_id=WORKSPACE_ID
).with_data(DATASET_ID).with_concurrency(2).with_evaluators(
    "Bias",
    MyCustomEvaluator(
        pass_fail_criteria={
            "Evaluation 1": PassFailCriteria(
                for_testrun_overall_pass_if=PassFailCriteriaForTestrunOverall(
                    ">", 3, "average"
                ),
                on_each_entry_pass_if=PassFailCriteriaOnEachEntry(">", 1),
            ),
            "Evaluation 2": PassFailCriteria(
                for_testrun_overall_pass_if=PassFailCriteriaForTestrunOverall(
                    overall_should_be="!=", value=2, for_result="average"
                ),
                on_each_entry_pass_if=PassFailCriteriaOnEachEntry("=", True),
            ),
        }
    ),
).with_workflow_id(
    WORKFLOW_ID
).run()

Validating test run config...
Sanitizing data...
Sanitizing evaluators...
Verifying if Bias is added to the workspace..
Creating test run: Local evaluators from SDK
You can view your test run here: https://app.getmaxim.ai/workspace/cln4nw1n80000mc3wqqwk4j0z/testrun/cm8eomjwi00nrycsfancrsu7h
You can safely quit this session or wait to see the final output in console.
Waiting for test run to complete...
Polling interval: 15 seconds
┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                Test run status: QUEUED                                                │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ totalEntries: 0 | runningEntries: 0 | queuedEntries: 0 | failedEntries: 0 | completedEntries: 0 | stoppedEntries: 0   │
└─────────────────────────────────────────────────────────────────────────────

RunResult(test_run_result=TestRunResult(link='https://app.getmaxim.ai/workspace/cln4nw1n80000mc3wqqwk4j0z/testrun/cm8eomjwi00nrycsfancrsu7h', result=[TestRunResultObj(name='Local evaluators from SDK', individual_evaluator_mean_score={'Bias': EvaluatorMeanScore(score=0, out_of=1, is_pass=True), 'Evaluation 1': EvaluatorMeanScore(score=1, out_of=None, is_pass=False), 'Evaluation 2': EvaluatorMeanScore(score=0, out_of=None, is_pass=True)}, usage=TestRunTokenUsage(total=None, input=None, completion=None), cost=TestRunCost(total=0, input=0, completion=0), latency=TestRunLatency(min=3586, max=12031.6804, p50=7299, p90=12031, p95=12031, p99=12031, mean=8003.25, standard_deviation=3057.8634, total=4))]), failed_entry_indices=[])